# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234883238735                   -0.50    7.0         
  2   -7.250352587576       -1.81       -1.41    1.0   9.81ms
  3   -7.251173811121       -3.09       -2.07    2.0   11.0ms
  4   -7.251122717325   +   -4.29       -1.99    2.0   11.9ms
  5   -7.251330560221       -3.68       -2.67    1.0   9.68ms
  6   -7.251338334098       -5.11       -3.30    1.0   9.43ms
  7   -7.251338712555       -6.42       -3.56    3.0   13.5ms
  8   -7.251338783611       -7.15       -3.95    1.0   9.90ms
  9   -7.251338795741       -7.92       -4.57    1.0   10.2ms
 10   -7.251338798367       -8.58       -4.68    3.0   13.8ms
 11   -7.251338798600       -9.63       -4.98    1.0   10.2ms
 12   -7.251338798692      -10.04       -5.52    2.0   12.2ms
 13   -7.251338798704      -10.95       -6.05    2.0   12.1ms
 14   -7.251338798704      -12.13       -6.32    2.0   12.6ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.035572741334516336
[ Info: Arnoldi iteration step 2: normres = 0.516352682394916
[ Info: Arnoldi iteration step 3: normres = 0.3494784023311286
[ Info: Arnoldi iteration step 4: normres = 1.0009113927748197
[ Info: Arnoldi iteration step 5: normres = 0.4437273556986637
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.65e-01, 7.94e-02, 2.49e-01, 2.41e-01, 2.06e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4223661686637928
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.98e-02, 1.62e-01, 3.53e-01, 8.91e-02, 6.27e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08315938489172774
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.34e-03, 2.14e-02, 1.73e-02, 2.99e-02, 6.45e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11870490046203669
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.24e-0